In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features, reduce_mem_usage, elo_save_feature
from preprocessing import get_dummies
import datetime

from tqdm import tqdm
import time
import sys
from joblib import Parallel, delayed

key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

In [2]:
# df_train = utils.read_df_pkl('../input/train0*')
# df_train.set_index(key, inplace=True)
# df_test = utils.read_df_pkl('../input/test0*')
# df_test.set_index(key, inplace=True)

df_hist = pd.read_csv('../input/historical_transactions.csv')
# df_new = pd.read_csv('../input/new_merchant_transactions.csv')

df_hist['purchase_amount_new'] = np.round(df_hist['purchase_amount'] / 0.00150265118 + 497.06, 2)
# df_new['purchase_amount_new'] = np.round(df_new['purchase_amount'] / 0.00150265118 + 497.06, 2)

df_hist['installments'] = df_hist['installments'].map(lambda x:  
                                                    1 if x<1 else
                                                    1 if x>100 else
                                                      x
                                                     )
# df_new['installments'] = df_new['installments'].map(lambda x:  
#                                                     1 if x<1 else
#                                                     1 if x>100 else
#                                                       x
#                                                      )

In [3]:
#========================================================================
# Dataset Load 
use_cols = [key, 'authorized_flag', 'installments', 'merchant_category_id', 'merchant_id', 'subsector_id', 'month_lag', 'purchase_amount_new', 'purchase_date']
df_hist = df_hist[use_cols]
# df_new = df_new[use_cols]

auth1 = df_hist[df_hist['authorized_flag']=='Y']
# auth0 = df_hist[df_hist['authorized_flag']=='N']

# df_trans = pd.concat([auth1, df_new], axis=0)

def get_new_columns(name,aggs):
    if len(name):
        return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]
    else:
        return [k + '_' + agg for k in aggs.keys() for agg in aggs[k]]
#========================================================================

In [9]:
#========================================================================
# Merchant id 別の集計
# month_lagで集計する（purchase_dateは別にやる）
# df = auth1

# def raddar_top_freq_merchant_agg(df, agg_term, new_max, new_min, old_max, old_min):
#========================================================================
df = auth1.head(100000)
new_max, new_min, old_max, old_min = 0,-1,-2,-3
agg_term = -20
# Args Setting
level = [key, 'merchant_id', 'month_lag']
# merchantの利用頻度を集計する期間を絞る（month_lagの値）
new_month_lag_max = new_max
new_month_lag_min = new_min
old_month_lag_max = old_max
old_month_lag_min = old_min
#========================================================================

#========================================================================
# Aggregation
print("Aggregation Start!!")

aggs = {}
aggs['purchase_amount_new'] = ['sum']
aggs['installments'] = ['mean', 'max', 'sum']
df_agg = df.groupby(level).agg(aggs)

new_cols = get_new_columns(name='', aggs=aggs)
df_agg.columns = new_cols
df_agg[f'purchase_amount_new_sum_per_installments_sum'] = df_agg[f'purchase_amount_new_sum'] / df_agg[f'installments_sum']

mer_cnt = df.groupby([key, 'merchant_id'])['month_lag'].nunique().reset_index().rename(columns={'month_lag':'month_lag_cnt'})
df_agg = df_agg.reset_index().merge(mer_cnt, how='inner', on=[key, 'merchant_id'])
#========================================================================

#========================================================================
# month_lag別に切り出して集計を行う
print("Aggregate Term Setting")
df_merchant = df_agg[df_agg['month_lag_cnt']>1]
del df_agg
gc.collect()
df_merchant.drop(['month_lag_cnt'], axis=1, inplace=True)

# 期間を絞る
new_term = new_month_lag_max - new_month_lag_min
old_term = old_month_lag_max - old_month_lag_min
new = df_merchant[df_merchant['month_lag']<= new_month_lag_max][df_merchant['month_lag']>= new_month_lag_min]
old = df_merchant[df_merchant['month_lag']<= old_month_lag_max][df_merchant['month_lag']>= old_month_lag_min]

feat_cols = [col for col in df_merchant.columns if col.count('amount') or col.count('install')]
aggs = {}
for col in feat_cols:
    if col.count('install') and not(col.count('per')):
        aggs[col] = ['mean']
    else:
        aggs[col] = ['sum']
        
# 複数month_lagをもつデータの場合は、集計する
if new_term>0:
    new.groupby([key, 'merchant_id'])[feat_cols]
else:
    new.set_index([key, 'merchant_id'], inplace=True)

if old_term>0:
    old.groupby([key, 'merchant_id'])[feat_cols]
else:
    old.set_index([key, 'merchant_id'], inplace=True)
#========================================================================
    
new.reset_index(inplace=True)
old.reset_index(inplace=True)
#========================================================================
# oldに存在するがnewにいないcard_id, merchantをnewにもたせる。
print("Get Lost Merchant and Card ID")
new['flg'] = 1
tmp_cols = [key, 'merchant_id']
old_lost = old[tmp_cols].merge(new[tmp_cols + ['flg']], how='left', on=[key, 'merchant_id'])
old_lost = old_lost[old_lost['flg'].isnull()]
old_lost = old_lost[tmp_cols]
new = pd.concat([new, old_lost], ignore_index=True)
new.drop('flg', axis=1, inplace=True)
#========================================================================
    
#========================================================================
# Make Ratio Feature
print("Make Ratio Feature")
fname = f'flag{new_month_lag_max}_{new_month_lag_min}-plag{old_month_lag_max}_{old_month_lag_min}'
new = new.merge(old, how='left', on=[key, 'merchant_id'])
for col in feat_cols:
   new[f"{fname}_{col}"]  = new[col+'_x'] / new[col+'_y']
   new[f"{fname}_{col}"].fillna(0, inplace=True)
#========================================================================

#========================================================================
# card_id * merchant_id別のtop frequency ranking
# all term version
if debug:
    df_hist = df_hist.head(1000000)
df_term = df_hist[df_hist['month_lag']>=agg_term]
mer_cnt = df_term.groupby([key, 'merchant_id'])['month_lag'].nunique().reset_index().rename(columns={'month_lag':'month_lag_cnt'})
mer_cnt.sort_values(by=[key, 'month_lag_cnt'], ascending=False, inplace=True)
mer_cnt = utils.row_number(df=mer_cnt, level=key)
mer_cnt.set_index([key, 'merchant_id'], inplace=True)

df_merchant = new.set_index(tmp_cols).join(mer_cnt).reset_index()


Aggregation Start!!
Aggregate Term Setting
Get Lost Merchant and Card ID
Make Ratio Feature


In [14]:
new.set_index(tmp_cols)

index_x  installments_max_x  \
card_id         merchant_id                                    
C_ID_0046db9f8a M_ID_06626b531d     10.0                 1.0   
                M_ID_0fd259ab85     13.0                 1.0   
                M_ID_4b51c0f9b3     31.0                 1.0   
                M_ID_5344303ae4     33.0                 1.0   
                M_ID_5344303ae4     34.0                 1.0   
                M_ID_9e84cda3b1     66.0                 3.0   
                M_ID_a6ca6aec72     70.0                 3.0   
C_ID_011b0d9794 M_ID_9139332ccc    119.0                 4.0   
                M_ID_9139332ccc    119.0                 4.0   
                M_ID_9139332ccc    120.0                 6.0   
                M_ID_9139332ccc    120.0                 6.0   
C_ID_01904d743d M_ID_00a6ca8a8a    129.0                 1.0   
                M_ID_00a6ca8a8a    129.0                 1.0   
                M_ID_39432481b2    138.0                 1.0   
                M_ID_bf31cbd73d    151.0                 1.0   
C_ID_01b098ff01 M_ID_00a6ca8a8a    162.0                 1.0   
                M_ID_00a6ca8a8a    163.0                 1.0   
                M_ID_5e1eb51cce    191.0                 1.0   
                M_ID_72d720df70    198.0                 1.0   
                M_ID_7ffe7268cd    208.0                 1.0   
                M_ID_e5374dabc0    239.0                 1.0   
                M_ID_e5374dabc0    240.0                 1.0   
                M_ID_ec1003ed6f    244.0                 1.0   
                M_ID_f86439cec0    250.0                 1.0   
                M_ID_f86439cec0    251.0                 1.0   
C_ID_02c2ed9fc0 M_ID_00a6ca8a8a    261.0                 1.0   
                M_ID_00a6ca8a8a    262.0                 1.0   
                M_ID_0136f79f8d    270.0                 1.0   
                M_ID_0136f79f8d    270.0                 1.0   
                M_ID_1d527e9ebf    279.0                 1.0   
                M_ID_1d527e9ebf    280.0                 1.0   
                M_ID_27f1c9c0de    285.0                 1.0   
                M_ID_2866c5358c    290.0                 1.0   
                M_ID_3a232be738    299.0                 1.0   
                M_ID_548f18cdd2    317.0                 1.0   
                M_ID_548f18cdd2    317.0                 1.0   
                M_ID_548f18cdd2    318.0                 1.0   
                M_ID_548f18cdd2    318.0                 1.0   
                M_ID_569cb2eef9    320.0                 1.0   
                M_ID_62b578c70a    323.0                 1.0   
                M_ID_7d2dd8800c    335.0                 1.0   
                M_ID_ac78c7d82a    360.0                 1.0   
                M_ID_ac78c7d82a    360.0                 1.0   
                M_ID_ac78c7d82a    361.0                 1.0   
                M_ID_ac78c7d82a    361.0                 1.0   
                M_ID_b3eadbc447    369.0                 1.0   
                M_ID_b3eadbc447    369.0                 1.0   
                M_ID_b3eadbc447    370.0                 1.0   
                M_ID_b3eadbc447    370.0                 1.0   
                M_ID_e4129b5119    383.0                 1.0   
                M_ID_e4129b5119    384.0                 1.0   
                M_ID_ec24d672a3    390.0                 1.0   
                M_ID_ec24d672a3    390.0                 1.0   
                M_ID_edc0990cd0    392.0                 1.0   
                M_ID_f2045dd267    394.0                 1.0   
                M_ID_f2045dd267    395.0                 1.0   
C_ID_02cc9ba024 M_ID_0c7b8adebc    410.0                 1.0   
                M_ID_142cb0b7e9    421.0                 1.0   
                M_ID_142cb0b7e9    422.0                 1.0   
                M_ID_1bd1f2e1c0    431.0                 1.0   
                M_ID_424af08136    449.0                 1.0   
 

In [7]:
del new, old, mer_cnt
gc.collect()

use_cols = [key, 'merchant_id'] + [col for col in df_merchant.columns if col.count('flag')] + ['month_lag_cnt', 'row_no']
df_merchant = df_merchant[use_cols]
#========================================================================


#========================================================================
# merchant_id別に集計を行ったら、
# 1. それらを更に集計する. frequencyが高いmerchantのみで集計するパターンも作る
# 2. frequencyの高いmerchnatでまとめてtop1~10カラムを作る ------->>> frequencyについては、全体と直近半年の両パターンでカウントし、特徴を作る
#========================================================================

feat_cols = [col for col in df_merchant.columns if col.count('flag')]

#========================================================================
# Top10のfrequency merchantをまとめてカラムにする
df_merchant = df_merchant[df_merchant['row_no']<=10]
df_merchant = df_merchant.set_index([key, 'row_no'])[feat_cols].unstack()
# Rename
fname = 'auth1_all'
df_merchant.columns =  [f"{fname}_top-M{col[1]}_merchant_{col[0]}" for col in df_merchant.columns]

#========================================================================

#========================================================================
# Save Feature
if agg_term<-13:
    prefix = '244_rad'
else:
    prefix = f'245_rad_min_Mlag{agg_term}'
    
print(f"{prefix} Feature Saving...")

base = utils.read_df_pkl('../input/base_no_out*')
base = base[[key, target]].set_index(key)
base = base.join(df_merchant)
base.fillna(-1, inplace=True)
del df_merchant
gc.collect()

elo_save_feature(df_feat=base, prefix=prefix)
print('Complete!')
#========================================================================

,card_id,merchant_id,flag0_-1-plag-2_-3_purchase_amount_new_sum,flag0_-1-plag-2_-3_installments_mean,flag0_-1-plag-2_-3_installments_max,flag0_-1-plag-2_-3_installments_sum,flag0_-1-plag-2_-3_purchase_amount_new_sum_per_installments_sum,month_lag_cnt,row_no
0,C_ID_00007093c1,M_ID_5357547c3e,0.000000,0.000000,0.000000,0.000000,0.000000,3,5
1,C_ID_00007093c1,M_ID_5357547c3e,0.000000,0.000000,0.000000,0.000000,0.000000,3,5
2,C_ID_00007093c1,M_ID_57df19bf28,0.847458,1.000000,1.000000,0.666667,1.271186,10,2
3,C_ID_00007093c1,M_ID_57df19bf28,2.000000,1.000000,1.000000,2.000000,1.000000,10,2
4,C_ID_00007093c1,M_ID_57df19bf28,0.711864,1.000000,1.000000,0.666667,1.067797,10,2
5,C_ID_00007093c1,M_ID_57df19bf28,1.680000,1.000000,1.000000,2.000000,0.840000,10,2
7,C_ID_00007093c1,M_ID_69423b34e4,0.000000,0.000000,0.000000,0.000000,0.000000,7,3
8,C_ID_00007093c1,M_ID_84e60960a3,0.948767,0.833333,0.833333,0.833333,1.138520,3,7
9,C_ID_00007093c1,M_ID_9400cf2342,0.368686,1.000000,1.000000,0.333333,1.106059,11,1
10,C_ID_00007093c1,M_ID_a33355a1b7,0.000000,0.000000,0.000000,0.000000,0.000000,3,8


In [26]:
pattern_list = [
    # One Month Ver
#     [0, 0, -1, -1]
    [-1, -1, -2, -2]
    ,[-2, -2, -3, -3]
    
    # Two Month Ver
    ,[0, -1, -2, -3]
    ,[-1, -2, -3, -4]
    ,[-2, -3, -4, -5]
    
    # Three Month Ver
    ,[0, -2, -3, -5]
    ,[-1, -3, -4, -6]
    ,[-2, -4, -5, -7]
    
    # One / Two Month Ver
    ,[0, 0, -1, -2]
    ,[-1, -1, -2, -3]
    ,[-2, -2, -3, -4]
    
    # One / Three Month Ver
    ,[0, 0, -1, -3]
    ,[-1, -1, -2, -4]
    ,[-2, -2, -3, -5]
    
    # One / Six Month Ver
    ,[0, 0, -1, -6]
    ,[-1, -1, -2, -7]
    ,[-2, -2, -3, -8]
    
    # Two / Three Month Ver
    ,[0, -1, -2, -4]
    ,[-1, -2, -3, -5]
    ,[-2, -3, -4, -6]
    
    # Two / Six Month Ver
    ,[0, -1, -2, -7]
    ,[-1, -2, -3, -8]
    ,[-2, -3, -4, -9]
    
    # Three / Six Month Ver
    ,[0, -2, -3, -8]
    ,[-1, -3, -4, -9]
    ,[-2, -4, -5, -10]
    
]
term_list = [-20, -6]
for agg_term in term_list:
    for pattern in pattern_list:
        raddar_top_freq_merchant_agg(auth1, agg_term, *pattern)

-20
-1 -1


,card_id,authorized_flag,installments,merchant_category_id,merchant_id,subsector_id,month_lag,purchase_amount_new,purchase_date
0,C_ID_4e6213e9bc,Y,1,80,M_ID_e020e9b302,37,-8,29.00,2017-06-25 15:33:07
1,C_ID_4e6213e9bc,Y,1,367,M_ID_86ec983688,16,-7,9.17,2017-07-15 12:10:45
2,C_ID_4e6213e9bc,Y,1,80,M_ID_979ed661fc,37,-6,17.65,2017-08-09 22:04:29
3,C_ID_4e6213e9bc,Y,1,560,M_ID_e6d5ae8ea6,34,-5,7.69,2017-09-02 10:06:26
4,C_ID_4e6213e9bc,Y,1,80,M_ID_e020e9b302,37,-11,16.00,2017-03-10 01:14:19


SystemExit: 